# Regression in torch

In [1]:
import torch
from torch import nn, optim

from torch.utils.data import TensorDataset, DataLoader

## Create network

You need to create a network object that describes the sequence of transformations performed by the model under consideration.

In [2]:
net = nn.Sequential(
    nn.Linear(in_features = 100, out_features = 10),
    nn.ReLU(),
    nn.Linear(in_features = 10, out_features = 1)
)
net

Sequential(
  (0): Linear(in_features=100, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=1, bias=True)
)

## Optimizer

Is a special object that allows you to change the weights of the network to make it better.

In [3]:
optim.Adam(net.parameters())

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

## Loss

In torch there are classes that implement loss functions.

In [4]:
loss = nn.MSELoss()
loss(torch.rand(100), torch.rand(100))

tensor(0.1757)

## Dataset

There are classes that implement a convenient way of organising your data sets. The advantage is that you just pass a sequence of tensors describing your objects and just by using the indexing syntax you can access any object you want.

In [5]:
dataset = TensorDataset(
    torch.rand(100, 2), 
    torch.rand(100, 1), 
    torch.rand(100, 5)
)
dataset[5]

(tensor([0.5017, 0.2694]),
 tensor([0.7112]),
 tensor([0.3316, 0.2062, 0.4393, 0.0639, 0.5028]))

## Dataloader

Object that implements the partitioning of the dataset into batches. You just pass the dataset object and the required batch size. By iterating on the gotten object you will get batches.

In [6]:
my_loader = DataLoader(
    TensorDataset(
        torch.rand(10, 3),
        torch.rand(10)
    ),
    batch_size = 2
)

for a,b in my_loader:
    print(a,b)

tensor([[0.9273, 0.7325, 0.2644],
        [0.8303, 0.4975, 0.5738]]) tensor([0.9972, 0.5394])
tensor([[0.4124, 0.5600, 0.0088],
        [0.5078, 0.0942, 0.3956]]) tensor([0.5644, 0.7418])
tensor([[0.7397, 0.4297, 0.1283],
        [0.0926, 0.3512, 0.6297]]) tensor([0.4397, 0.8969])
tensor([[0.6764, 0.2055, 0.4628],
        [0.3138, 0.7287, 0.8516]]) tensor([0.5232, 0.4896])
tensor([[0.6208, 0.2415, 0.4725],
        [0.4048, 0.2538, 0.6035]]) tensor([0.8369, 0.3332])


## Fit model

Now lets combine all previous steps to show simple model fit.

Sample generation.

In [7]:
n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2

Fitting algorithm.

In [8]:
data_loader = DataLoader(TensorDataset(X, Y), batch_size = 20)
net = nn.Sequential(
    nn.Linear(in_features = n_features, out_features = 1, bias = False)
)
optimizer = optim.Adam(net.parameters(), lr = 0.1)
loss_fn = nn.MSELoss()
loss_values = []

for x, y in data_loader:
    
    optimizer.zero_grad()
    # forward path
    output = net(x)
    # loss compution
    loss_val = loss_fn(output.ravel(), y)
    loss_values.append(loss_val.item())
    # packward path
    loss_val.backward()
    optimizer.step()

print("Loss path ", "->".join([str(round(val, 2)) for val in loss_values]))
print("True weights " , w_true)
print("Estimated weights", list(net.parameters())[0][0])

Loss path  1.58->0.94->1.17->0.52->0.39->0.23->0.34->0.34->0.22->0.24->0.29->0.33->0.43->0.2->0.42
True weights  tensor([-0.7069, -0.2510])
Estimated weights tensor([-0.9058, -0.2342], grad_fn=<SelectBackward0>)
